In [ ]:
!pip install implicit

     |████████████████████████████████| 18.5 MB 2.9 MB/s 


In [ ]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np
import scipy.sparse as sparse
import pandas as pd
from sklearn.preprocessing import *
master_path = "/content/drive/MyDrive/Applied Machine Learning Project/h-and-m-personalized-fashion-recommendations"
# pp_path = "/content/drive/MyDrive/Applied Machine Learning Project/preprocessed_data"
csv_users = master_path+"/customers.csv"
csv_items = master_path+"/articles.csv"
csv_train = master_path+"/transactions_train.csv"

/usr/local/lib/python3.7/dist-packages/implicit/gpu/__init__.py:14: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: no CUDA-capable device is detected (/project/./implicit/gpu/utils.h:71)'
  f"CUDA extension is built, but disabling GPU support because of '{e}'",


In [ ]:
users_df = pd.read_csv(csv_users)
articles_df = pd.read_csv(csv_items, dtype={'article_id': str})


In [ ]:
train_df = pd.read_csv(csv_train,parse_dates=['t_dat'])


In [ ]:
customer_list_unique = users_df['customer_id'].unique().tolist()
article_list_unique = articles_df['article_id'].unique().tolist()
user_ids = dict(list(enumerate(customer_list_unique)))
item_ids = dict(list(enumerate(article_list_unique)))

In [ ]:
user_map = {u: uidx for uidx, u in user_ids.items()}
item_map = {i: iidx for iidx, i in item_ids.items()}

,Unnamed: 0,user_id,article_id,price,total_item,user_ID,article_ID
0,0,0,176209023,0.035576,1,0,99
1,1,0,568601006,0.050831,2,0,15980
2,2,0,568601043,0.050831,1,0,16000
3,3,0,607642008,0.012695,1,0,23967
4,4,0,625548001,0.044051,1,0,29482
...,...,...,...,...,...,...,...
28109844,28109844,1371978,840360003,0.013542,1,1362279,89664
28109845,28109845,1371978,866755002,0.043203,1,1362279,95240
28109846,28109846,1371978,866755002,0.050831,1,1362279,95240
28109847,28109847,1371978,882810001,0.016932,1,1362279,98746


In [ ]:
len(ALL_ITEMS)

105542

In [ ]:
# inspired by 
# https://towardsdatascience.com/building-a-collaborative-filtering-recommender-system-with-clickstream-data-dffc86c8c65
class recommender(object):
  def __init__(self, user_df, articles_df, df_for_rec, use_GPU=False):
    print("Users reading....")
    self.users_df = users_df
    self.users = users_df['customer_id'].unique().tolist()
    self.items = articles_df['article_id'].unique().tolist()
    self.user_ids = dict(list(enumerate(self.users)))
    self.item_ids = dict(list(enumerate(self.items)))
    self.user_map = {u: uidx for uidx, u in self.user_ids.items()}
    self.item_map = {i: iidx for iidx, i in self.item_ids.items()}

    print("Users loaded!")
    print("Articles reading....")
    self.use_GPU = use_GPU
    self.articles_df = articles_df

    print("Articles loaded!")
    print("Training data reading...")
    self.df_for_rec = df_for_rec 
    print("Training data loaded!")
    self.df_for_rec["user_id"] = self.df_for_rec["user_id"].astype("category")
    self.df_for_rec["article_id"] = self.df_for_rec["article_id"].astype("category")
    # self.df_for_rec["user_ID"] = self.df_for_rec["user_id"].cat.codes
    
    # self.df_for_rec["article_ID"] = self.df_for_rec["article_id"].cat.codes
    print("Creating Sparse user-items matrix")
    self.sparse_users_items = sparse.csr_matrix((self.df_for_rec['total_item'].astype(float), 
                                       (self.df_for_rec['user_ID'], self.df_for_rec['article_ID'])))
    print("Done!")
    print("Creating Sparse items-user matrix")
    self.sparse_items_users = sparse.csr_matrix((self.df_for_rec['total_item'].astype(float), 
                                       (self.df_for_rec['article_ID'],self.df_for_rec['user_ID'])))
    print("Done!")

    #article has 12 feature, customer estimatley has 6 and transaction data has 5
    os.environ['OPENBLAS_NUM_THREADS']='1'
    os.environ['KMP_DUPLICATE_LIB_OK']='True'
    os.environ['MKL_NUM_THREADS']='1'
    self.als_model = AlternatingLeastSquares(factors=360, regularization=0.01, 
                                             use_gpu=use_GPU, iterations=5, 
                                             dtype=np.float32, 
                                             calculate_training_loss=True)
  
  def train(self, alpha = 15):
    data = (self.sparse_items_users *alpha).astype("double")
    print("Training model")
    self.als_model.fit(data)
    print("Model trained!")
  
  def get_article_info(self, top_idx):
    article_ids = [self.item_ids[i] for i in top_idx]
    return self.articles_df[self.articles_df['article_id'].isin(article_ids)]
  def get_user_info(self, top_idx):
    user_ids = [ self.customer_ids[i] for i in top_idx]
    return self.users_df[self.users_df['customer_id'].isin(user_ids)]

  def get_recommendations(self, user_idx, top_n=10):
    recommendations = self.als_model.recommend(user_idx,
                             self.sparse_users_items[user_idx],
                             N=top_n)
    
    recs = pd.DataFrame({"item_idx":recommendations[0], "score":recommendations[1]})
    return recs

  def get_similar_items(self, item_idx, top_n=10):
    similars = self.als_model.similar_items(item_idx,N=top_n)
    recs = pd.DataFrame({"item_idx":similars[0], "score":similars[1]})
    return recs
  
  def get_similar_users(self, user_idx, top_n=10):
    similars = self.als_model.similar_users(user_idx,N=top_n)
    recs = pd.DataFrame({"user_idx":similars[0], "score":similars[1]})
    return recs

In [ ]:
rec = recommender(users_df, articles_df, df_for_rec)

Users reading....
Users loaded!
Articles reading....
Articles loaded!
Training data reading...
Training data loaded!
Creating Sparse user-items matrix
Done!
Creating Sparse items-user matrix
Done!


In [ ]:
rec.train()

Training model


  0%|          | 0/5 [00:00<?, ?it/s]

Model trained!


In [ ]:
recs = rec.get_recommendations(1)

In [ ]:
recs["item_idx"].values

array([1243740,  645029,  324005,  694880, 1215377,  760856,  444064,
        492616,  369330,  486965], dtype=int32)

In [ ]:
[rec.item_ids[i] for i in recs["item_idx"].values]

KeyError: ignored

In [ ]:
rec.articles_df["article_id"]

0         0108775015
1         0108775044
2         0108775051
3         0110065001
4         0110065002
             ...    
105537    0953450001
105538    0953763001
105539    0956217002
105540    0957375001
105541    0959461001
Name: article_id, Length: 105542, dtype: object

In [ ]:
rec.df_for_rec["article_id"][0:10]

0    176209023
1    568601006
2    568601043
3    607642008
4    625548001
5    627759010
6    656719005
7    694736004
8    697138006
9    745232001
Name: article_id, dtype: category
Categories (104547, int64): [108775015, 108775044, 108775051, 110065001, ..., 952938001, 953450001,
                             953763001, 956217002]